## Reference

https://www.tensorflow.org/programmers_guide/variables

In [1]:
import tensorflow as tf

## Creating a Variable

> Unlike tf.Tensor objects, a **tf.Variable exists outside the context of a single session.run** call.

> Internally, a tf.Variable stores a persistent tensor. Specific ops allow you to read and modify the values of this tensor. These modifications are visible across multiple tf.Sessions, so multiple workers can see the same values for a tf.Variable.

> The best way to create a variable is to call the tf.get_variable function. tf.get_variable also allows you to reuse a previously created variable of the same name, making it easy to define models which reuse layers.

In [40]:
tf.reset_default_graph()

my_variable = tf.get_variable("my_variable", [1, 2, 3])
print(my_variable)

my_int_variable = tf.get_variable("my_int_variable", 
                                  [1, 2, 3], 
                                  dtype=tf.int32, 
                                  initializer=tf.zeros_initializer)
print(my_int_variable)

<tf.Variable 'my_variable:0' shape=(1, 2, 3) dtype=float32_ref>
<tf.Variable 'my_int_variable:0' shape=(1, 2, 3) dtype=int32_ref>


## Initializing Variables

tf.get_variable() can not only access existing variables but can also create new variables

In [44]:
with tf.Session() as sess:
    print(sess.run(tf.report_uninitialized_variables()))
    
    # initialize
    sess.run(tf.global_variables_initializer())
    
    print(sess.run(tf.report_uninitialized_variables()))    

[b'my_variable' b'my_int_variable']
[]


In [76]:
tf.reset_default_graph()

const_1 = tf.constant_initializer(value=1)

with tf.variable_scope("vs"):
    # use tf.Variable(10, [1], name="v") doesn't work
    v = tf.get_variable(name="v", shape=[1], initializer=const_1)
    print(v.name)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
    
with tf.variable_scope("vs", reuse=True):
    v = tf.get_variable("v")
    print(v.name)       
    print(v.eval(session=sess))

sess.close()
    

vs/v:0
vs/v:0
[ 1.]


## name_scope

In [24]:
tf.reset_default_graph()

const_1 = tf.constant_initializer(value=1)

with tf.name_scope('ns'):
    # get_variable(name, shape=None, dtype=None, ...)
    v1 = tf.get_variable("v1", shape=[1], dtype=tf.float32, initializer=const_1) # get_variable ignores name_scope    
    v2 = tf.Variable([2.0], name='v2') # tf.Variable(<initial-value>, name=<optional-name>)
    v3 = v1 + v2      

    v21 = tf.Variable([2.0], name='v2')  # tensorflow append _1 for this variable
    v22 = tf.Variable([2.0], name='v2')  # tensorflow append _2 for this variable
    
print(v1.name)
print(v2.name)
print(v3.name)
print(v21.name)
print(v22.name)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print(sess.run(v1))
    print(sess.run(v2))
    print(sess.run(v3))    

v1:0
ns/v2:0
ns/add:0
ns/v2_1:0
ns/v2_2:0
[ 1.]
[ 2.]
[ 3.]


## variable_scope
### scope.reuse_variables()

In [28]:
tf.reset_default_graph()

const_1 = tf.constant_initializer(value=1)

with tf.variable_scope('vs') as scope:
    # get_variable(name, shape=None, dtype=None, ...)
    v1 = tf.get_variable("v1", shape=[1], dtype=tf.float32, initializer=const_1) # get_variable ignores name_scope    
    
    scope.reuse_variables()
    v1_reuse = tf.get_variable("v1")
    
print(v1.name)
print(v2.name)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    print(sess.run(v1))
    print(sess.run(v1_reuse))

vs/v1:0
ns/v2:0
[ 1.]
[ 1.]


## Combination of Scopes

Note that name scopes do not affect the names of variables.

In [82]:
tf.reset_default_graph()

const_1 = tf.constant_initializer(value=1)

with tf.name_scope('ns'):
    with tf.variable_scope('vs'):
        nvv = tf.get_variable("nvv", shape=[1], dtype=tf.float32, initializer=const_1)
        nv_add = nvv + 10

        print(nvv.name)    # variable is affected by variable_scope only
        print(nv_add.name) # op is affected by both variable_scope and name_scope


with tf.variable_scope('vs'):
    with tf.name_scope('ns'):
        vnv = tf.get_variable("vnv", shape=[1], dtype=tf.float32, initializer=const_1)

        print(vnv.name) # variable is affected by variable_scope only

with tf.variable_scope('vs'):
    with tf.variable_scope('vs'):
        vvv = tf.get_variable("vvv", shape=[1], dtype=tf.float32, initializer=const_1)

        print(vvv.name) # variable is affected by nested variable_scope
        

vs/nvv:0
ns/vs/add:0
vs/vnv:0
vs/vs/vvv:0
